# 📊 Top One Model - Análise do Dataset Interno

## Objetivo
Este notebook realiza a análise exploratória inicial do dataset interno de crédito com 700.000+ registros para entender:
- Estrutura e qualidade dos dados disponíveis
- Features internas existentes 
- Distribuições e padrões nos dados
- Identificação de variáveis geográficas para associação com dados macro regionais

## Estrutura do Projeto
```
├── data/
│   ├── internal_data/          # Dataset interno (.xlsx)
│   ├── external/              # Dados de web scraping
│   └── processed/             # Dados processados
├── src/
│   ├── data_collection/       # Módulos de web scraping
│   ├── data_engineering/      # Engenharia de features
│   ├── modeling/              # Modelos de ML
│   ├── validation/            # Validação e testes
│   └── prediction/            # Sistema de predição
└── notebooks/                 # Análises exploratórias
```

**Autores:** Pedro Schuves Marodin, Enzo Holtzmann Gaio  
**Data:** Agosto 2025

## 1. Environment Setup and Library Imports

In [ ]:
# Environment Setup
import sys
import os
sys.path.append('../src')

# Data Manipulation and Analysis
import pandas as pd
import numpy as np
from scipy import stats

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Web Scraping Libraries (for later use)
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

# Machine Learning Libraries 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import lightgbm as lgb

# Configuration and Utilities
import yaml
import warnings
warnings.filterwarnings('ignore')

# Display Settings
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Todas as bibliotecas importadas com sucesso!")
print(f"📍 Diretório atual: {os.getcwd()}")
print(f"🐍 Versão Python: {sys.version}")
print(f"🐼 Versão Pandas: {pd.__version__}")
print(f"📊 Versão NumPy: {np.__version__}")

## 2. Data Loading and Initial Exploration

Vamos carregar o dataset interno e fazer uma primeira análise da estrutura dos dados.

In [ ]:
# Definir caminho para o dataset interno
DATASET_PATH = "../data/internal_data/dataset_interno_top_one.xlsx"

try:
    # Carregar o dataset
    print("📥 Carregando dataset interno...")
    df_internal = pd.read_excel(DATASET_PATH)
    
    print(f"✅ Dataset carregado com sucesso!")
    print(f"📊 Dimensões: {df_internal.shape[0]:,} linhas x {df_internal.shape[1]} colunas")
    print(f"💾 Tamanho em memória: {df_internal.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    
except FileNotFoundError:
    print(f"❌ Arquivo não encontrado em: {DATASET_PATH}")
    print("📝 Por favor, certifique-se de que o arquivo está no local correto!")
    df_internal = None
except Exception as e:
    print(f"❌ Erro ao carregar o dataset: {str(e)}")
    df_internal = None

In [ ]:
# Análise inicial do dataset se carregado com sucesso
if df_internal is not None:
    print("=" * 60)
    print("📊 ANÁLISE INICIAL DO DATASET")
    print("=" * 60)
    
    # Informações básicas
    print(f"📋 Colunas disponíveis ({len(df_internal.columns)}):")
    for i, col in enumerate(df_internal.columns, 1):
        print(f"  {i:2d}. {col}")
    
    print(f"\n📈 Primeiras 5 linhas do dataset:")
    display(df_internal.head())
    
    print(f"\n📊 Informações sobre tipos de dados:")
    display(df_internal.info())
    
    print(f"\n🔍 Estatísticas descritivas:")
    display(df_internal.describe())
else:
    print("⏳ Aguardando carregamento do dataset...")

## 3. Análise Geográfica e Identificação de Cidades

Análise específica para identificar colunas geográficas que serão usadas para associação com dados regionais coletados via web scraping.

In [ ]:
# Análise geográfica - identificar colunas de cidade/município
if df_internal is not None:
    print("🌍 ANÁLISE GEOGRÁFICA")
    print("=" * 50)
    
    # Buscar colunas que podem conter informações geográficas
    geo_keywords = ['cidade', 'municipio', 'city', 'municipality', 'local', 'endereco', 'cep']
    possible_geo_cols = []
    
    for col in df_internal.columns:
        if any(keyword in col.lower() for keyword in geo_keywords):
            possible_geo_cols.append(col)
    
    print(f"📍 Colunas geográficas identificadas: {possible_geo_cols}")
    
    # Analisar cada coluna geográfica
    for col in possible_geo_cols:
        print(f"\n🔍 Análise da coluna: {col}")
        print(f"   - Tipo: {df_internal[col].dtype}")
        print(f"   - Valores únicos: {df_internal[col].nunique()}")
        print(f"   - Valores faltantes: {df_internal[col].isnull().sum()} ({df_internal[col].isnull().mean()*100:.1f}%)")
        
        if df_internal[col].dtype == 'object':
            # Mostrar algumas cidades mais frequentes
            top_cities = df_internal[col].value_counts().head(10)
            print(f"   - Top 10 cidades:")
            for city, count in top_cities.items():
                print(f"     • {city}: {count:,} registros")
    
    # Se encontrou colunas geográficas, criar visualização
    if possible_geo_cols:
        city_col = possible_geo_cols[0]  # Usar primeira coluna encontrada
        
        # Distribuição geográfica
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        top_20_cities = df_internal[city_col].value_counts().head(20)
        top_20_cities.plot(kind='bar')
        plt.title(f'Top 20 Cidades - {city_col}')
        plt.xticks(rotation=45, ha='right')
        plt.ylabel('Número de Registros')
        
        plt.subplot(1, 2, 2)
        # Distribuição percentual
        city_distribution = df_internal[city_col].value_counts()
        others_count = city_distribution[20:].sum() if len(city_distribution) > 20 else 0
        
        plot_data = city_distribution.head(10).copy()
        if others_count > 0:
            plot_data['Outras'] = others_count
        
        plt.pie(plot_data.values, labels=plot_data.index, autopct='%1.1f%%')
        plt.title('Distribuição Geográfica (Top 10 + Outras)')
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n📊 Resumo geográfico:")
        print(f"   - Total de cidades únicas: {df_internal[city_col].nunique()}")
        print(f"   - Cidade mais frequente: {df_internal[city_col].mode().iloc[0]} ({df_internal[city_col].value_counts().iloc[0]:,} registros)")
        print(f"   - Cobertura geográfica: {(1-df_internal[city_col].isnull().mean())*100:.1f}%")
    
    else:
        print("⚠️ Nenhuma coluna geográfica clara identificada.")
        print("💡 Sugestão: Verifique se existe coluna com informações de localização.")
else:
    print("⏳ Dataset não carregado ainda.")